In [1]:
import gymnasium
from gymnasium import spaces
import numpy as np
import pandas as pd
import heapq
import matplotlib.pyplot as plt
from collections import defaultdict
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
import os

class RadarTaskSchedulerEnv(gymnasium.Env):
    def __init__(self, task_file="radar_tasks_with_standalone_searches.csv"):
        super(RadarTaskSchedulerEnv, self).__init__()

        self.original_tasks_df = pd.read_csv(task_file)
        # Store a mapping of Target_ID and Task_Type to their tasks for easy lookup
        self.target_task_map = defaultdict(lambda: defaultdict(list))
        for index, row in self.original_tasks_df.iterrows():
            self.target_task_map[row['Target_ID']][row['Task_Type']].append(row.to_dict())

        self.num_unique_targets = len(self.original_tasks_df['Target_ID'].unique())

        self.priority_queue = [] # This will hold tasks currently available for scheduling
        self.entry_count = 0 # To handle ties in priority
        self.current_task_data = None # The task that was just popped for processing

        self.scheduled_tasks_log = [] # Log of all tasks processed in the current episode
        self.dropped_tasks_in_episode = 0 # Track dropped tasks

        # --- Dynamic Stage Tracking for each Target ---
        # Key: Target_ID, Value: {'current_stage': 'Search', 'search_count': 0, ...}
        self.target_progression_states = {}

        # --- Configuration for stage progression ---
        self.MAX_STAGES_PER_TARGET_TYPE = {
            'Search': 1,
            'Detection': 4,
            'Tracking': 5,
            'Classification': 6
        }
        self.STAGE_ORDER = ['Search', 'Detection', 'Tracking', 'Classification', 'Locked']

        # Observation Space: [Duration, Deadline, Init_Power]
        # Max values are rough estimates based on typical ranges in the dataset.
        self.observation_space = spaces.Box(low=0, high=100, shape=(3,), dtype=np.float32)

        # Action Space: 3 (Delay: 0, 1, 2) * 3 (Compress: 0, 1, 2) * 3 (Radar Choice: 0, 1, 2) = 27 actions
        self.action_space = spaces.Discrete(3 * 3 * 3)

        self._last_info = {}
        self._last_reward = 0

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)

        self.priority_queue = []
        self.entry_count = 0
        self.scheduled_tasks_log = []
        self.dropped_tasks_in_episode = 0
        self._last_info = {}
        self._last_reward = 0

        # Reset target progression states and initialize with 'Search' tasks
        self.target_progression_states = {}
        initial_tasks_to_queue = []

        # Find all unique Target_IDs and their first 'Search' task based on min Request_Time
        for target_id in self.original_tasks_df['Target_ID'].unique():
            search_tasks_for_target = [t for t in self.target_task_map[target_id]['Search']]
            if search_tasks_for_target:
                # Find the 'Search' task with the minimum Request_Time for this target
                initial_search_task = min(search_tasks_for_target, key=lambda x: x['Request_Time'])
                initial_tasks_to_queue.append(initial_search_task)

                # Initialize the progression state for this target
                self.target_progression_states[target_id] = {
                    'current_stage_in_env': 'Search', # The highest stage this target can currently generate tasks for
                    'Search_count': 0, # How many Search tasks we've _attempted_ to schedule/process for this target
                    'Detection_count': 0,
                    'Tracking_count': 0,
                    'Classification_count': 0
                }
                # Increment the count for the initial search task we're adding
                self.target_progression_states[target_id]['Search_count'] += 1
            else:
                # Handle targets without a 'Search' task if that's a possibility
                print(f"Warning: Target_ID {target_id} has no 'Search' tasks in the dataset.")


        # Push initial search tasks onto the priority queue
        # Sort by Request_Time to simulate tasks arriving over time
        initial_tasks_to_queue.sort(key=lambda x: x['Request_Time'])

        for task_dict in initial_tasks_to_queue:
            # Priority is negative for min-heap behavior on max priority
            negative_priority = -task_dict['Priority']
            heapq.heappush(self.priority_queue, (negative_priority, self.entry_count, task_dict))
            self.entry_count += 1
        
        obs = self._get_observation()
        return obs, {}

    def _generate_next_task_for_target(self, target_id, last_processed_task_info):
        """
        Generates the next logical task for a target based on its progression state
        and whether it met criteria for the previous stage.
        """
        target_state = self.target_progression_states.get(target_id)
        if not target_state or target_state['current_stage_in_env'] == 'Locked':
            return None # Target has completed its progression or doesn't exist

        current_stage = target_state['current_stage_in_env']
        current_stage_count = target_state[f'{current_stage}_count']

        next_stage_type = current_stage # Default: continue with the same stage type

        # Check if criteria for advancing to the next stage are met
        # NOTE: In a real environment, 'last_processed_task_info' would need to indicate
        # success/failure, but for dataset generation, we assume success to generate tasks.
        # Here, we *must* assume the RL agent made a successful decision for progression.
        if current_stage in self.MAX_STAGES_PER_TARGET_TYPE and \
           current_stage_count >= self.MAX_STAGES_PER_TARGET_TYPE[current_stage]:
            
            current_stage_index = self.STAGE_ORDER.index(current_stage)
            if current_stage_index + 1 < len(self.STAGE_ORDER):
                potential_next_stage = self.STAGE_ORDER[current_stage_index + 1]
                
                if potential_next_stage == 'Locked':
                    target_state['current_stage_in_env'] = 'Locked'
                    return None # Target is now locked

                next_stage_type = potential_next_stage
                target_state['current_stage_in_env'] = next_stage_type # Update env's tracking

        # Find available tasks in the original dataset for this (Target_ID, next_stage_type)
        available_tasks = self.target_task_map[target_id][next_stage_type]
        
        # Filter tasks that have not been "generated" (i.e., had their stage_count fulfilled) yet
        # and have a Stage_Count matching what we need next.
        # This is a bit tricky: We need to find the specific task from the dataset that
        # corresponds to the *next* logical stage count for this target.
        # E.g., if a target just finished 'Detection_4', it needs 'Tracking_1' now.
        required_stage_count = target_state[f'{next_stage_type}_count'] + 1 # +1 because we're looking for the *next* one

        potential_next_task = None
        for task in available_tasks:
            if task['Stage_Count'] == required_stage_count:
                potential_next_task = task
                break
        
        if potential_next_task:
            target_state[f'{next_stage_type}_count'] += 1 # Increment the count as we are *generating* this task
            return potential_next_task
        
        return None # No more tasks for this target/stage in the dataset, or criteria not met


    def step(self, action_index):
        if not self.priority_queue:
            return np.array([0, 0, 0], dtype=np.float32), 0, True, False, {
                "task_dropped": False,
                "task_id": "N/A",
                "message": "No tasks left to process.",
                "dropped_tasks_count": self.dropped_tasks_in_episode,
                "completed_tasks_count": len(self.scheduled_tasks_log) - self.dropped_tasks_in_episode
            }

        _priority, _entry_count, task_to_process = heapq.heappop(self.priority_queue)

        delay = action_index // 9 # 0, 1, 2
        compress = (action_index % 9) // 3 # 0, 1, 2
        radar_choice = action_index % 3 # 0, 1, 2

        # # Apply actions to task parameters
        # tns = task_to_process['Request_Time'] + delay
        # Pn = task_to_process['Init_Power'] + compress

        # task_dropped = (tns > task_to_process['Deadline']) or \
        #                (Pn > task_to_process['Max_Power'])

        # reward = 0.0

        # Calculate TNS (Task Nominal Start Time)
        # The new approach includes 'Duration' in the deadline check, but for calculating 'tns' itself,
        # it's still based on 'Request_Time' + delay. The variable name is updated to 'calculated_tns'
        tns = task_to_process['Request_Time'] + delay

        # Calculate Pn (Power Nominal) based on the compress action
        power_increase_from_compression = 0.0
        if compress == 0:
            power_increase_from_compression = 0.0
        elif compress == 1:
            power_increase_from_compression = 0.15 * task_to_process['Init_Power']
        elif compress == 2:
            power_increase_from_compression = 0.30 * task_to_process['Init_Power']

        Pn = task_to_process['Init_Power'] + power_increase_from_compression

        # Check for task drop conditions
        task_dropped = False
        # The deadline check now includes the 'Duration' of the task
        if (tns + task_to_process['Duration']) > task_to_process['Deadline']:
            task_dropped = True
        # The power check remains similar but uses 'calculated_Pn'
        if Pn > task_to_process['Max_Power']:
            task_dropped = True

        reward = 0.0

        if not task_dropped:
            reward += 1.0 # Base reward for successful completion

            # Reward for meeting deadline early or with margin
            time_to_deadline_margin = task_to_process['Deadline'] - tns
            if time_to_deadline_margin >= 0:
                reward += time_to_deadline_margin * 0.01 # Small bonus for time margin
            else: # Late but not dropped (if possible, though logic usually drops)
                reward -= 0.2

            # Reward for power efficiency (lower power used)
            power_margin = task_to_process['Max_Power'] - Pn
            if power_margin >= 0:
                reward += power_margin * 0.005 # Small bonus for power margin
            else: # Over power but not dropped (if possible)
                reward -= 0.2

            # Bonus for minimal delay/compression if still successful
            if delay == 0:
                reward += 0.05
            if compress == 0:
                reward += 0.05
            
            # Special reward for 'locking' a target (reaching Classification_6 and successfully completing)
            target_id = task_to_process['Target_ID']
            current_stage = task_to_process['Task_Type']
            stage_count = task_to_process['Stage_Count']

            if current_stage == 'Classification' and stage_count == self.MAX_STAGES_PER_TARGET_TYPE['Classification']:
                # This task completes the 'Classification' sequence for this target
                reward += 10.0 # Significant bonus for locking a target
                # Mark target as locked in environment's state to prevent further task generation for it
                self.target_progression_states[target_id]['current_stage_in_env'] = 'Locked'

        else:
            reward -= 1.0 # Penalty for dropping a task
            self.dropped_tasks_in_episode += 1
            # Larger penalties for critical drops
            if tns > task_to_process['Deadline'] + 5 or Pn > task_to_process['Max_Power'] + 5:
                reward -= 0.5


        info = {
            "task_id": task_to_process["Task_ID"],
            "target_id": task_to_process["Target_ID"],
            "task_type": task_to_process["Task_Type"],
            "stage_count": task_to_process["Stage_Count"],
            "task_dropped": task_dropped,
            "delay_action": delay,
            "compress_action": compress,
            "radar_choice_action": radar_choice,
            "calculated_tns": tns,
            "calculated_Pn": Pn,
            "original_deadline": task_to_process['Deadline'],
            "original_max_power": task_to_process['Max_Power']
        }
        self._last_info = info
        self._last_reward = reward

        # Log the processed task
        self.scheduled_tasks_log.append({
            'Target_ID': task_to_process['Target_ID'],
            'Task_ID': task_to_process['Task_ID'],
            'Task_Type': task_to_process['Task_Type'],
            'Priority': task_to_process['Priority'],
            'Action_Delay': delay,
            'Action_Compress': compress,
            'Action_Radar': radar_choice,
            'Dropped': task_dropped,
            'Final_TNS': tns,
            'Final_Pn': Pn,
            'Original_Deadline': task_to_process['Deadline'],
            'Original_Max_Power': task_to_process['Max_Power'],
            'Stage_Count': task_to_process['Stage_Count']
        })

        # --- Dynamic Task Generation (Crucial for the new scenario) ---
        # If the task was successfully processed, try to generate the next task for this target
        if not task_dropped: # Only progress if the current task was not dropped
            next_task_for_target = self._generate_next_task_for_target(
                task_to_process['Target_ID'],
                info # Pass info about the processed task if needed by _generate_next_task_for_target
            )
            if next_task_for_target:
                negative_priority = -next_task_for_target['Priority']
                heapq.heappush(self.priority_queue, (negative_priority, self.entry_count, next_task_for_target))
                self.entry_count += 1
        
        # Check if episode is done: all unique targets have reached 'Locked' state AND priority queue is empty
        all_targets_locked = all(state['current_stage_in_env'] == 'Locked' for state in self.target_progression_states.values())
        done = all_targets_locked and not self.priority_queue

        if done:
            info['dropped_tasks_count'] = self.dropped_tasks_in_episode
            info['completed_tasks_count'] = len(self.scheduled_tasks_log) - self.dropped_tasks_in_episode
        else:
            info['dropped_tasks_count'] = self.dropped_tasks_in_episode # Keep track of running count
            info['completed_tasks_count'] = len(self.scheduled_tasks_log) - self.dropped_tasks_in_episode


        next_obs = self._get_observation()
        return next_obs, reward, done, False, info

    def _get_observation(self):
        if self.priority_queue:
            # The observation should be based on the HIGHEST PRIORITY task currently available
            # We don't remove it, just peek
            _priority, _entry_count, task_dict = self.priority_queue[0]
            self.current_task_data = task_dict # Store for internal use if needed
            return np.array([task_dict['Duration'], task_dict['Deadline'], task_dict['Init_Power']], dtype=np.float32)
        else:
            # If no tasks, return a zero observation, indicating an empty queue
            self.current_task_data = None
            return np.array([0, 0, 0], dtype=np.float32)

    def close(self):
        pass

# --- Episode Logger Callback (No Changes Needed) ---
# This callback remains the same as it correctly logs episode rewards and dropped tasks
# by accessing info from the environment.
class EpisodeLoggerCallback(BaseCallback):
    def __init__(self, output_file, total_episodes_target, verbose=0):
        super(EpisodeLoggerCallback, self).__init__(verbose)
        self.output_file = output_file
        self.total_episodes_target = total_episodes_target
        self.episode_num = 0
        self.reward_log = []
        self.task_drop_log = []
        self.completed_tasks_log = []

    def _on_step(self) -> bool:
        if self.locals['dones'][0]:
            self.episode_num += 1
            
            last_step_info = self.locals['infos'][0]
            
            # Monitor wrapper adds 'episode' key to info dict
            if 'episode' in last_step_info:
                episode_stats = last_step_info['episode']
                total_reward_episode = episode_stats['r']
                
                task_drops_episode = last_step_info.get('dropped_tasks_count', 0)
                completed_tasks_episode = last_step_info.get('completed_tasks_count', 0)
                
                self.reward_log.append(total_reward_episode)
                self.task_drop_log.append(task_drops_episode)
                self.completed_tasks_log.append(completed_tasks_episode)

                epsilon = self.model.exploration_rate if hasattr(self.model, 'exploration_rate') else 'N/A'

                log_message = (
                    f"Episode {self.episode_num}/{self.total_episodes_target} "
                    f"— Total Reward: {total_reward_episode:.2f} "
                    f"— Tasks Dropped: {task_drops_episode} "
                    f"— Tasks Completed: {completed_tasks_episode} "
                    f"— Epsilon (approx): {epsilon:.3f}"
                )
                print(log_message)
                self.output_file.write(log_message + "\n")
        return True

    def _on_rollout_end(self) -> None:
        pass

    def _on_training_end(self) -> None:
        pass

# --- Main Training and Evaluation Script ---
# --- Main Training and Evaluation Script ---
if __name__ == "__main__":
    # Ensure the new dataset is generated first
    # (assuming the dataset generation script was run and created 'dynamic_radar_tasks_full_progression.csv')
    
    # Create the base environment first
    base_env = RadarTaskSchedulerEnv(task_file="radar_tasks_with_standalone_searches.csv")
    # Wrap it with Monitor
    env = Monitor(base_env) # Monitor wraps the environment to log episode statistics

    dqn_hyperparams = {
        "learning_rate": 0.0005,
        "buffer_size": 20000, # Increased buffer size for potentially more tasks
        "learning_starts": 1000, # Increased learning starts
        "batch_size": 64, # Increased batch size
        "gamma": 0.99, # Adjusted gamma
        "train_freq": (4, "step"), # Train every 4 steps
        "gradient_steps": 1,
        "target_update_interval": 200, # Adjusted target update interval
        "exploration_fraction": 0.2, # Slightly longer exploration
        "exploration_initial_eps": 1.0,
        "exploration_final_eps": 0.05,
        "max_grad_norm": 10,
        "seed": 42
    }

    model = DQN("MlpPolicy", env, verbose=0, **dqn_hyperparams, device="auto")

    # The number of tasks per episode is now dynamic, equal to the total number of tasks in the dataset
    # that can possibly be generated (including progression).
    # A single episode now means processing ALL possible tasks for ALL targets until they are 'Locked'.
    # So, total_timesteps will be roughly (num_targets * avg_tasks_per_target_lifecycle) * num_episodes
    
    # Calculate approx total tasks if all targets go through all stages
    # Access these attributes from the base_env (the unwrapped environment)
    approx_tasks_per_full_target_lifecycle = sum(base_env.MAX_STAGES_PER_TARGET_TYPE.values())
    total_tasks_in_dataset_potential = base_env.num_unique_targets * approx_tasks_per_full_target_lifecycle

    episodes_to_train = 500 # Train for a fixed number of "full lifecycle" episodes
    total_timesteps_for_training = episodes_to_train * total_tasks_in_dataset_potential # Each step processes one task

    output_filename = "dqn_learning_results_dynamic.txt"
    
    with open(output_filename, "w") as f_out:
        f_out.write("--- DQN Training Log (Dynamic Tasks) ---\n\n")
        print("Starting DQN training with dynamic task generation...")
        f_out.write("Starting DQN training with dynamic task generation...\n\n")

        episode_logger_callback = EpisodeLoggerCallback(
            output_file=f_out,
            total_episodes_target=episodes_to_train
        )

        model.learn(total_timesteps=total_timesteps_for_training, callback=episode_logger_callback)

        print("\nTraining complete!")
        f_out.write("\nTraining complete!\n")

        model_path = "dqn_radar_scheduler_dynamic_model"
        model.save(model_path)
        print(f"Final model saved to {model_path}")
        f_out.write(f"Final model saved to {model_path}\n")

        # --- Plotting Results ---
        reward_log = episode_logger_callback.reward_log
        task_drop_log = episode_logger_callback.task_drop_log
        completed_tasks_log = episode_logger_callback.completed_tasks_log

        window_size = 50 # For moving average

        # Plot Rewards
        if reward_log:
            plt.figure(figsize=(14,7))
            plt.plot(reward_log, label='Episode Rewards', color='orange', alpha=0.7, linewidth=1)
            if len(reward_log) >= window_size:
                moving_avg = np.convolve(reward_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg = range(window_size - 1, len(reward_log))
                plt.plot(x_moving_avg, moving_avg, label=f'Moving Average ({window_size} episodes)', color='blue', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Total Reward', fontsize=12)
            plt.title('Total Reward over Episodes (DQN - Dynamic Tasks)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_reward_plot_dynamic.png")
            plt.close()
            print(f"Reward plot saved as dqn_reward_plot_dynamic.png")
            f_out.write(f"Reward plot saved as dqn_reward_plot_dynamic.png\n")
        else:
            print("\nReward log is empty. No reward plot will be generated.")
            f_out.write("\nReward log is empty. No reward plot will be generated.\n")

        # Plot Tasks Dropped
        if task_drop_log:
            plt.figure(figsize=(14,7))
            plt.plot(task_drop_log, label='Tasks Dropped per Episode', color='red', alpha=0.8, linewidth=1)
            if len(task_drop_log) >= window_size:
                moving_avg_drops = np.convolve(task_drop_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_drops = range(window_size - 1, len(task_drop_log))
                plt.plot(x_moving_avg_drops, moving_avg_drops, label=f'Moving Average ({window_size} episodes)', color='darkred', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Tasks Dropped', fontsize=12)
            plt.title('Number of Tasks Dropped per Episode (DQN - Dynamic Tasks)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_task_drop_plot_dynamic.png")
            plt.close()
            print(f"Task drop plot saved as dqn_task_drop_plot_dynamic.png")
            f_out.write(f"Task drop plot saved as dqn_task_drop_plot_dynamic.png\n")
        else:
            print("\nTask drop log is empty. No task drop plot will be generated.")
            f_out.write("\nTask drop log is empty. No task drop plot will be generated.\n")

        # Plot Tasks Completed
        if completed_tasks_log:
            plt.figure(figsize=(14,7))
            plt.plot(completed_tasks_log, label='Tasks Completed per Episode', color='green', alpha=0.8, linewidth=1)
            if len(completed_tasks_log) >= window_size:
                moving_avg_completed = np.convolve(completed_tasks_log, np.ones(window_size)/window_size, mode='valid')
                x_moving_avg_completed = range(window_size - 1, len(completed_tasks_log))
                plt.plot(x_moving_avg_completed, moving_avg_completed, label=f'Moving Average ({window_size} episodes)', color='darkgreen', linewidth=2)
            plt.xlabel('Episode', fontsize=12)
            plt.ylabel('Tasks Completed', fontsize=12)
            plt.title('Number of Tasks Completed per Episode (DQN - Dynamic Tasks)', fontsize=14)
            plt.legend(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig("dqn_task_completed_plot_dynamic.png")
            plt.close()
            print(f"Task completed plot saved as dqn_task_completed_plot_dynamic.png")
            f_out.write(f"Task completed plot saved as dqn_task_completed_plot_dynamic.png\n")
        else:
            print("\nTask completed log is empty. No task completed plot will be generated.")
            f_out.write("\nTask completed log is empty. No task completed plot will be generated.\n")


        f_out.write("\n\n--- Running Final Evaluation Episode for Schedule and Metrics ---\n")
        print("\n--- Running Final Evaluation Episode for Schedule and Metrics ---")

        # Use a fresh environment for evaluation
        eval_env = RadarTaskSchedulerEnv(task_file="radar_tasks_with_standalone_searches.csv")
        
        obs, _ = eval_env.reset()
        done = False
        
        final_eval_scheduled_tasks = [] # Store tasks from this final eval run

        while not done:
            action_index, _states = model.predict(obs, deterministic=True) # Use deterministic actions for evaluation
            obs, reward, done, _, info = eval_env.step(action_index)
            final_eval_scheduled_tasks.append(eval_env._last_info) # Log info from each step

        f_out.write("\n--- Final Task Schedule (from last evaluation) ---\n")
        print("\n--- Final Task Schedule (from last evaluation) ---")
        if final_eval_scheduled_tasks:
            schedule_df = pd.DataFrame(eval_env.scheduled_tasks_log) # Use the log from eval_env
            f_out.write(schedule_df.to_string(index=False))
            print(schedule_df.to_string(index=False))
        else:
            f_out.write("No tasks processed in the final evaluation. This might mean the environment did not start a full episode.\n")
            print("No tasks processed in the final evaluation. This might mean the environment did not start a full episode.")


        f_out.write("\n\n--- Performance Metrics (from last evaluation) ---\n")
        print("\n--- Performance Metrics (from last evaluation) ---")

        if eval_env.scheduled_tasks_log:
            total_tasks_attempted = len(eval_env.scheduled_tasks_log)
            completed_tasks = sum(1 for task in eval_env.scheduled_tasks_log if not task['Dropped'])
            dropped_tasks_count = sum(1 for task in eval_env.scheduled_tasks_log if task['Dropped'])
            locked_targets_count = sum(1 for state in eval_env.target_progression_states.values() if state['current_stage_in_env'] == 'Locked')


            if total_tasks_attempted > 0:
                percent_tasks_completed = (completed_tasks / total_tasks_attempted) * 100
                percent_tasks_dropped = (dropped_tasks_count / total_tasks_attempted) * 100
            else:
                percent_tasks_completed = 0.0
                percent_tasks_dropped = 0.0

            metrics = {
                "Total Unique Targets in Dataset": eval_env.num_unique_targets,
                "Targets Successfully Locked (Completed all stages)": locked_targets_count,
                "Total Tasks Attempted (in last evaluation)": total_tasks_attempted,
                "Tasks Successfully Completed (Individual)": completed_tasks,
                "Tasks Dropped (Individual)": dropped_tasks_count,
                "Percentage of Tasks Completed (Individual)": f"{percent_tasks_completed:.2f}%",
                "Percentage of Tasks Dropped (Individual)": f"{percent_tasks_dropped:.2f}%"
            }

            metrics_df = pd.DataFrame(metrics.items(), columns=['Metric', 'Value'])
            f_out.write(metrics_df.to_string(index=False))
            print(metrics_df.to_string(index=False))
        else:
            f_out.write("No metrics could be calculated from the final evaluation as no tasks were processed.\n")
            print("No metrics could be calculated from the final evaluation as no tasks were processed.")

    env.close()
    eval_env.close()
    print("All environments closed.")

Starting DQN training with dynamic task generation...
Episode 1/500 — Total Reward: 70.02 — Tasks Dropped: 50 — Tasks Completed: 106 — Epsilon (approx): 0.998
Episode 2/500 — Total Reward: 55.20 — Tasks Dropped: 51 — Tasks Completed: 94 — Epsilon (approx): 0.996
Episode 3/500 — Total Reward: 50.65 — Tasks Dropped: 51 — Tasks Completed: 90 — Epsilon (approx): 0.995
Episode 4/500 — Total Reward: 61.48 — Tasks Dropped: 51 — Tasks Completed: 99 — Epsilon (approx): 0.993
Episode 5/500 — Total Reward: 61.77 — Tasks Dropped: 51 — Tasks Completed: 102 — Epsilon (approx): 0.991
Episode 6/500 — Total Reward: 65.53 — Tasks Dropped: 51 — Tasks Completed: 102 — Epsilon (approx): 0.989
Episode 7/500 — Total Reward: 68.85 — Tasks Dropped: 50 — Tasks Completed: 106 — Epsilon (approx): 0.988
Episode 8/500 — Total Reward: 67.33 — Tasks Dropped: 50 — Tasks Completed: 105 — Epsilon (approx): 0.986
Episode 9/500 — Total Reward: 86.54 — Tasks Dropped: 50 — Tasks Completed: 120 — Epsilon (approx): 0.984
Epis